#### Agrego una feature con datos extraidos de una página (https://es.investing.com/currencies/usd-mxn-historical-data) para recopilar informacion sobre la divisa norteamericana, agregando la devaluacion misma junto con el dolar del día.

In [267]:
import pandas as pd
import numpy as np

In [268]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [269]:
path_features_train = "features_csvs/train_con_dolar_y_devaluacion.csv"
path_features_test = "features_csvs/test_con_dolar_y_devaluacion.csv"

In [270]:
datos_historicos_dolar = pd.read_csv("../utils/DatoshistoricosUSD_MXN.csv")

datos_historicos_dolar.Fecha = pd.to_datetime(datos_historicos_dolar.Fecha)
datos_historicos_dolar = datos_historicos_dolar.drop(['Apertura','Máximo','Mínimo'] ,axis = 1)
datos_historicos_dolar = datos_historicos_dolar.rename(columns = {'Fecha':'fecha','Último':'dolar','% var.':'varianza_dolar'})

datos_historicos_dolar.dolar = datos_historicos_dolar.dolar.str.replace(",",".").astype(float)

datos_historicos_dolar.varianza_dolar = datos_historicos_dolar.varianza_dolar.str.replace(",",".")
datos_historicos_dolar.varianza_dolar = datos_historicos_dolar.varianza_dolar.str.replace("%","").astype(float)

datos_historicos_dolar = datos_historicos_dolar.loc[:,['fecha','dolar','varianza_dolar']]
datos_historicos_dolar = datos_historicos_dolar.sort_values('fecha')

In [271]:
train.fecha = pd.to_datetime(train.fecha)
train_with_dolar_info = pd.merge(train,datos_historicos_dolar,on = 'fecha',how='left')
train_with_dolar_info = train_with_dolar_info.loc[:,['id','fecha','dolar','varianza_dolar']].sort_values(by='fecha')
train_with_dolar_info = train_with_dolar_info[np.isfinite(train_with_dolar_info['id'])]
train_with_dolar_info = train_with_dolar_info.sort_values(by = 'fecha')
train_with_dolar_info.varianza_dolar.fillna(0, inplace=True)
train_with_dolar_info.dolar.fillna(method = 'ffill',inplace = True)
train_with_dolar_info.dolar.fillna(method = 'bfill',inplace = True)
train_with_dolar_info['devaluacion_acumulada'] = train_with_dolar_info.varianza_dolar.cumsum()
train_with_dolar_info = train_with_dolar_info.loc[:,['id','dolar','devaluacion_acumulada']]

In [272]:
test.fecha = pd.to_datetime(test.fecha)
test_with_dolar_info = pd.merge(test,datos_historicos_dolar,on = 'fecha',how='left')
test_with_dolar_info = test_with_dolar_info.loc[:,['id','fecha','dolar','varianza_dolar']].sort_values(by='fecha')
test_with_dolar_info = test_with_dolar_info[np.isfinite(train_with_dolar_info['id'])]
test_with_dolar_info = test_with_dolar_info.sort_values(by = 'fecha')
test_with_dolar_info.varianza_dolar.fillna(0, inplace=True)
test_with_dolar_info.dolar.fillna(method = 'ffill',inplace = True)
test_with_dolar_info.dolar.fillna(method = 'bfill',inplace = True)
test_with_dolar_info['devaluacion_acumulada'] = test_with_dolar_info.varianza_dolar.cumsum()
test_with_dolar_info = test_with_dolar_info.loc[:,['id','dolar','devaluacion_acumulada']]

/home/braianvillalba/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [273]:
train_with_dolar_info.to_csv(path_features_train, index=False)
test_with_dolar_info.to_csv(path_features_test, index=False)